<h1 align="center"> YRoots Tutorial </h1>

YRoots is a numerical rootfinding package that quickly and precisely finds and returns all of the real roots of a system of equations in a compact interval in $\mathbb{R}^n$.

Yroots is guaranteed to work as long as the functions entered are smooth and contiuous on the search interval and all roots in the search interval are simple and finite in number. Under these assumptions, YRoots can find and return the zeros of even complex systems of equations with several variables.

## YRoots Syntax

The YRoots solver takes as input a list of functions as well as a list of lower bounds together with a list of upper bounds defining a search interval. It returns a numpy array with each row containing the coordinates (in dimesion order) of the root. The syntax for calling the solve function is as follows:

```python
yr.solve(funcs, a, b)
```

where `funcs` is a list of $n$ callable functions in $n$ variables, `a` is a list of the $n$ lower bounds in each dimension, and `b` is a list of the correspondnig $n$ upper bounds. This tutorial contains several examples that demonstrate this syntax. 

## Set up YRoots

Using pip:
  ```
pip install git+https://github.com/tylerjarvis/RootFinding.git
  ```
  
In addition, you will need to have `numpy`, `numba`, and `scipy` installed in order to run YRoots. See the documentation for the corresponding packages to learn how to install any of these.

<u>Before proceeding in this tutorial, you will need to complete the above process and run the following import statements: </u>

In [3]:
import numpy as np
import yroots as yr

## Example 1: Bivariate System
Consider the following bivariate system of equations:

$$0 = \sin(xy) + x\log(y+3) - x^2 + \frac{1}{y-4}$$
$$6 = \cos(3xy) + e^{\frac{3y}{x-2}} - x.$$

Solutions of the system subject to the constrains $-1\leq x\leq0,-2\leq y\leq1$ are common roots of the functions

$$f(x,y) = \sin(xy) + x\log(y+3) - x^2 + \frac{1}{y-4} $$
$$g(x,y) = \cos(3xy) + e^{\frac{3y}{x-2}} - x - 6 $$
on the search domain $[-1,0]\times[-2,1]$.

To find the roots of this system, simply define the corresponding functions, lower bounds, and upper bounds in the correct format (lists or numpy arrays) and pass these as arguments to the solve function:

NOTE: <u> YRoots uses just in time compiling</u>, which means that part of the code will not be compiled until a system of functions to solve is given (rather than compiling all the code upon importing the module). Therefore, <u>the very first time a system of equations of a particular dimension is passed to the solver, the compilation will take several seconds before the solver actually begins to run.</u> To see the true speed of the solver, simply run the code again. This second iteration, and any other systems of equations of the same dimension, will run at the true speed after this first compilation.

In [5]:
f = lambda x,y : np.sin(x*y) + x*np.log(y+3) - x**2 + 1/(y-4)
g = lambda x,y : np.cos(3*x*y) + np.exp(3*y/(x-2)) - x - 6
a = [-1,-2] #lower bounds on x and y
b = [0,1] #upper bounds on x and y
roots = yr.solve([f,g], a, b)

print(roots)

[[-0.410034   -1.40471685]
 [-0.73720226 -1.65461673]]


To give a measure for the speed and accuracy of the search, the following cell of code prints the time taken for the search and the *residuals* of the roots, or the absolute difference between the actual function values at the computed roots and zero:

In [6]:
%time roots = yr.solve([f,g], a, b)

print('residuals for f are {}'.format(np.abs(f(roots[:,0],roots[:,1]))))
print('residuals for g are {}'.format(np.abs(g(roots[:,0],roots[:,1]))))

CPU times: total: 31.2 ms
Wall time: 49 ms
residuals for f are [2.77555756e-16 1.94289029e-16]
residuals for g are [8.88178420e-16 2.66453526e-15]


As you can see, using properties of Chebyshev polynomial interpolation, YRoots identified the two roots contained in the search interval very quickly and with near machine-epsilon precision.

## Example 2: Higher-dimensional System

Consider the problem of finding roots of the following 5-dimensional system of equations on the interval $[0,2\pi]^5$:

$$\cos(x_1) + x_5 = 1$$
$$\cos(x_2) + x_4 = 2$$
$$\cos(x_3) + x_3 = 3$$
$$\cos(x_4) + x_2 = 4$$
$$\cos(x_5) + x_1 = 5$$

Go ahead and test out the solver on this system of equations using the code below. The time taken and maximum residual value will also be printed.

Note also that for this problem, we have set the `verbose` option of the solver to True. This can be useful to track the progress of approximation and rootfinding, especially with systems of equations that are high-dimensional or more complex. You will see that short statements are outputted to the terminal, indicating the approximation results, the search interval, and the number of roots found.

NOTE: Since this is the first time we are working in this new dimension (dimension 5) in this notebook, the solver will need to compile new code before it can solve the problem, which will take several seconds. To see the true speed of the solver on this problem, simply rerun the code after the first iteration finishes.

In [8]:
#functions
f1 = lambda x1, x2, x3, x4, x5: np.cos(x1) + x5 - 1
f2 = lambda x1, x2, x3, x4, x5: np.cos(x2) + x4 - 2
f3 = lambda x1, x2, x3, x4, x5: np.cos(x3) + x3 - 3
f4 = lambda x1, x2, x3, x4, x5: np.cos(x4) + x2 - 4
f5 = lambda x1, x2, x3, x4, x5: np.cos(x5) + x1 - 5

#domain
a = [0]*5
b = [2*np.pi]*5

#solve
%time roots = yr.solve([f1,f2,f3,f4,f5],a,b,verbose=True)
print(roots)

#maximum residual
print(np.max([np.abs(f(*[roots[:,i] for i in range(5)])) for f in [f1,f2,f3,f4,f5]]))

# CPU times: total: 46.9 ms
# Wall time: 90.7 ms
# [[4.57744547 4.55389487 3.79438861 2.15783137 1.13453433]]
# 1.1102230246251565e-15

Approximation shapes: 0: (21, 1, 1, 1, 2) 1: (1, 21, 1, 2, 1) 2: (1, 1, 21, 1, 1) 3: (1, 4, 1, 21, 1) 4: (2, 1, 1, 1, 21)
Searching on interval [[0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586]]
Finding roots... *
Found 1 root

CPU times: total: 46.9 ms
Wall time: 86.3 ms
[[4.57744547 4.55389487 3.79438861 2.15783137 1.13453433]]
1.1102230246251565e-15


In higher dimensions, as the input functions become more complex, the time YRoots needs to find the roots of the system of functions increases as accurate approximation of such functions require manipulating many more values. However, when compared with other existing rootfinders, YRoots accurately solves such systems of equations with great speed relative.

As an illustration, the following code calls the solver on a more complicated version of the same system of equations solved above. For each function, an additional variable has been added to the cosine term. Try running the code below and watch the solver find the 49 common roots of this system of equations on the search interval, which should take around just 2-3 minutes on a typical computer. Note the low maximum residual value, which will also be printed:

In [9]:
f1 = lambda x1, x2, x3, x4, x5: np.cos(x1*x4) + x5 - 1
f2 = lambda x1, x2, x3, x4, x5: np.cos(x2*x5) + x4 - 2
f3 = lambda x1, x2, x3, x4, x5: np.cos(x3*x1) + x3 - 3
f4 = lambda x1, x2, x3, x4, x5: np.cos(x4*x2) + x2 - 4
f5 = lambda x1, x2, x3, x4, x5: np.cos(x5*x3) + x1 - 5

a = [0]*5
b = [2*np.pi]*5

%time roots = yr.solve([f1,f2,f3,f4,f5],a,b,verbose=True)
print(roots)

#maximum residual
print(np.max([np.abs(f(*[roots[:,i] for i in range(5)])) for f in [f1,f2,f3,f4,f5]]))

Approximation shapes: 0: (50, 1, 1, 50, 2) 1: (1, 50, 1, 2, 50) 2: (50, 1, 50, 1, 1) 3: (1, 50, 1, 50, 1) 4: (2, 1, 50, 1, 87)
Searching on interval [[0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586]]
Finding roots... *************************************************
Found 49 roots

CPU times: total: 1min 3s
Wall time: 2min 6s
[[5.95617903 3.21250323 2.22453203 1.74914246 1.54582102]
 [5.99988277 3.22442619 2.04451747 1.73674702 1.54408297]
 [4.01511593 3.00004825 3.97165144 2.09763587 1.53817437]
 [4.91121894 3.2535773  3.42813666 2.15495463 1.400553  ]
 [5.53092835 3.00184035 3.89071562 2.07289746 0.54758359]
 [5.56130621 3.00387896 3.98524302 2.06235932 0.54369572]
 [4.39710871 3.33966345 2.59537445 1.627007   0.35589322]
 [4.51240438 3.39523359 3.17105066 1.57923199 0.33473531]
 [4.18254226 3.72815455 2.7014153  1.33784667 0.22721903]
 [4.63980737 4.29935039 2.48614721 2.48678558 0.48360851]
 [4.65305679 4.6312599

## Example 3: Univariate System

The `yr.solve` method can also be used to quickly find the roots of a univariate function. In this case, `a` and `b` can simply be entered as floats, and `funcs` does not need to be a list.

As an example, find the zeros of $f(x) = \sin(e^{3x})$ on $[-1,2]$ using the code below:

In [11]:
f = lambda x : np.sin(np.exp(3*x))
%time roots = yr.solve(f, -1, 2,)

print(f"Number of roots: {len(roots)}")
print(f"\nMaximum residual: {np.max(np.abs(f(roots)))}")

CPU times: total: 141 ms
Wall time: 510 ms
Number of roots: 128

Maximum residual: 6.468437707466133e-13


## Example 4: Using MultiCheb and MultiPower Objects

When a function in a system is a polynomial, it may be useful to pass it in as a YRoots `MultiCheb` or `MultiPower` object, which corresponds to a multivariate polynomials in the Chebyshev basis or in the power basis (respectively). These objects may be more cumbersome to create, but they have a special `evaluate_grid` method which allows for faster approximation.

Polynomials in $n$-dimensions are represented by an $n$-dimensional array of coefficients. For a system with three variables, the $(i,j,k)$ spot in the coefficient tensor corresponds to the coefficients of $x^iy^jz^k$ in the power basis or $T_i(x)T_j(y)T_k(z)$ in the Chebyshev basis. It is usually easiest to construct this coefficient tensor by initializing a tensor of zeros and then setting each nonzero coefficient to the correct value.

For example, $f(x,y) = 5x^3 + 4 xy^2 + 3x^2 + 2y^2 + 1$ could be initialized as 

```python
coeff = np.zeros((4,4)) #4x4 matrix because it's a degree 3 polynomial
coeff[3,0] = 5
coeff[1,2] = 4
coeff[2,0] = 3
coeff[0,2] = 2
coeff[0,0] = 1
f = yr.MultiPower(coeff)
```
                         
Similarly, $g(x,y,z) = 5 T_2(x) + 3T_1(x)T_2(y) + 2$ would be initialized as

```python
coeff = np.zeros((3,3))
coeff[2,0] = 5
coeff[1,2] = 3
coeff[0,0] = 2
g = yr.MultiCheb(coeff)
```

Try out using these special YRoots objects in the following code:


In [12]:
coeff = np.zeros((4,4))
coeff[3,0], coeff[1,2], coeff[2,0], coeff[0,2], coeff[0,0] = 5, 4, 3, 2, 1
f = yr.MultiPower(coeff)

coeff = np.zeros((3,3))
coeff[2,0], coeff[1,2], coeff[0,0] = 5, 3, 2
g = yr.MultiCheb(coeff)

%time roots = yr.solve([f,g],[-1,-1],[1,1])
print(roots)
print("Maximum residuals: f: ", np.max(np.abs(f(roots))), " g: ", np.max(np.abs(g(roots))))

CPU times: total: 15.6 ms
Wall time: 23.1 ms
[[-0.69924092 -0.97485404]
 [-0.69924092  0.97485404]]
Maximum residuals: f:  4.440892098500626e-16  g:  1.3322676295501878e-15


## That's it!

This concludes the tutorial on YRoots. Feel free to try out the package using your own system of equations and interval in the space provided immediately below, or check out the documentation for more information.

In [ ]:
# Try out YRoots here.

In [ ]:
# Good luck and happy rootfinding :)